<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 6>Capstone Project code notebook</font></h1>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)


## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a **restaurant** in **Toronto**, Canada.

Here we will try to find if someone wants to open a new restaurant in the city, then which location is best suited for it keeping in mind the competitors and which income group of people will be attracted most to it based on the **population of the neighbourhood**.

Since there are lots of restaurants in Toronto we will try to detect **locations that are not already crowded with restaurants**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use data science tools and methodology to generate the most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* All existing restaurants in the neighborhood (any type of restaurant)
* Age group of people with their income
* Distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**

In [3]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

In [4]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr = pd.DataFrame(columns=column_names)

## 1. Download and Explore Dataset

In [ ]:
from urllib.request import urlopen
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(wiki)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

In [ ]:
Toronto=soup.find('table', class_='wikitable sortable')
Toronto

In [ ]:
#Generate lists
Pos=[]
Bor=[]
Neig=[]

for row in Toronto.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: 
        Pos.append(cells[0].find(text=True))
        Bor.append(cells[1].find(text=True))
        Neig.append(cells[2].find(text=True))

        
#Add Data to our DataFrame
Nebr['Postal_Code']=Pos
Nebr['Borough']=Bor
Nebr['Neighborhood']=Neig
Nebr

### Data Cleaning

If Borough is Not Assigned drop row.

Reset Index

In [ ]:
Nebr = Nebr.drop(Nebr[Nebr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)
Nebr.index = pd.RangeIndex(len(Nebr.index))
Nebr

In [9]:
Nebr.shape

(103, 3)

In [66]:
column_n = ['NEBRVAL'] 
NEBR_NA = pd.DataFrame(columns=column_n)
NA=['Not assigned'] 
NEBR_NA['NEBRVAL'] = NA

Nebr1=Nebr

for row_index,row in Nebr.iterrows():
    if((Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
       
        Nebr1.loc[row_index,['Neighborhood']] = Nebr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

Nebr1.head()

,Postal_Code,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


In [67]:
Nebr2=Nebr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

column_names = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr3 = pd.DataFrame(columns=column_names)

Nebr3 = Nebr2.drop(Nebr2[Nebr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
Nebr3.index = pd.RangeIndex(len(Nebr3.index))
Nebr3.head()

,Postal_Code,Borough,Neighborhood
0,M4E\n,East Toronto\n,The Beaches\n
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale\n"
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West\n"
3,M4M\n,East Toronto\n,Studio District\n
4,M4N\n,Central Toronto\n,Lawrence Park\n


In [68]:
column = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr_ungrp = pd.DataFrame(columns=column_names)

Nebr_ungrp = Nebr1.drop(Nebr1[Nebr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)


Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))
Nebr_ungrp.head()

,Postal_Code,Borough,Neighborhood
0,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
1,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
2,M5B\n,Downtown Toronto\n,"Garden District, Ryerson\n"
3,M5C\n,Downtown Toronto\n,St. James Town\n
4,M4E\n,East Toronto\n,The Beaches\n


In [13]:
#!conda install -c conda-forge geopy --yes 
import time
from geopy.geocoders import Nominatim

In [14]:
from geopy.util import get_version
get_version()

'2.0.0'

In [15]:
geolocator = Nominatim(scheme='http', user_agent="ES1234")

for row_index, item in Nebr_ungrp.iterrows():
    
    list1 = Nebr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    time.sleep(5)
    if(location is not None):
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Latitude'] = location.latitude
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Longitude'] = location.longitude
          

In [16]:
print(Nebr_ungrp)

   Postal_Code             Borough                                       Neighborhood   Latitude  Longitude
0        M5A\n  Downtown Toronto\n                        Regent Park, Harbourfront\n        NaN        NaN
1        M7A\n  Downtown Toronto\n      Queen's Park, Ontario Provincial Government\n        NaN        NaN
2        M5B\n  Downtown Toronto\n                         Garden District, Ryerson\n        NaN        NaN
3        M5C\n  Downtown Toronto\n                                   St. James Town\n  43.669403 -79.372704
4        M4E\n      East Toronto\n                                      The Beaches\n  43.671024 -79.296712
5        M5E\n  Downtown Toronto\n                                      Berczy Park\n  43.647984 -79.375396
6        M5G\n  Downtown Toronto\n                               Central Bay Street\n        NaN        NaN
7        M6G\n  Downtown Toronto\n                                         Christie\n  43.664111 -79.418405
8        M5H\n  Downtown Tor

In [17]:
import json 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
#!conda install -c conda-forge folium=0.5.0 --yes  #Uncomment if not installed
import folium 

Solving environment: | 
  - anaconda/linux-64::ca-certificates-2020.1.1-0, anaconda/linux-64::certifi-2019.11.28-py37_0, anaconda/linux-64::openssl-1.1.1d-h7b6447c_4
  - anaconda/linux-64::certifi-2019.11.28-py37_0, anaconda/linux-64::openssl-1.1.1d-h7b6447c_4, defaults/linux-64::ca-certificates-2020.1.1-0
  - anaconda/linux-64::ca-certificates-2020.1.1-0, anaconda/linux-64::openssl-1.1.1d-h7b6447c_4, defaults/linux-64::certifi-2019.11.28-py37_0
  - anaconda/linux-64::openssl-1.1.1d-h7b6447c_4, defaults/linux-64::ca-certificates-2020.1.1-0, defaults/linux-64::certifi-2019.11.28-py37_0
  - anaconda/linux-64::ca-certificates-2020.1.1-0, defaults/linux-64::certifi-2019.11.28-py37_0, defaults/linux-64::openssl-1.1.1d-h7b6447c_4
  - defaults/linux-64::ca-certificates-2020.1.1-0, defaults/linux-64::certifi-2019.11.28-py37_0, defaults/linux-64::openssl-1.1.1d-h7b6447c_4
  - anaconda/linux-64::ca-certificates-2020.1.1-0, anaconda/linux-64::certifi-2019.11.28-py37_0, defaults/linux-64::openssl-

In [18]:
print('We have {} boroughs and {} neighborhoods.'.format(
        len(Nebr_ungrp['Borough'].unique()),
        Nebr_ungrp.shape[0]
    )
)

Nebr_ungrp.dropna(inplace =True)
Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ES1234")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We have 4 boroughs and 39 neighborhoods.
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [19]:
Nebr_ungrp

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5C\n,Downtown Toronto\n,St. James Town\n,43.669403,-79.372704
1,M4E\n,East Toronto\n,The Beaches\n,43.671024,-79.296712
2,M5E\n,Downtown Toronto\n,Berczy Park\n,43.647984,-79.375396
3,M6G\n,Downtown Toronto\n,Christie\n,43.664111,-79.418405
4,M6H\n,West Toronto\n,"Dufferin, Dovercourt Village\n",43.660202,-79.435719
5,M4M\n,East Toronto\n,Studio District\n,43.649585,-79.390683
6,M4N\n,Central Toronto\n,Lawrence Park\n,43.729199,-79.403252
7,M5N\n,Central Toronto\n,Roselawn\n,43.709852,-79.404295
8,M4P\n,Central Toronto\n,Davisville North\n,43.697936,-79.397291
9,M6R\n,West Toronto\n,"Parkdale, Roncesvalles\n",43.641281,-79.451599


In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Nebr_ungrp['Latitude'], Nebr_ungrp['Longitude'], Nebr_ungrp['Borough'], Nebr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Lets use FOURSQUARE API to explore the neighbourhood

In [21]:
CLIENT_ID = 'HT2UDAD5MOUTTLYX000WZ55BUZYKPXDJGHHOYGK0XVVJHA5Q' # your Foursquare ID
CLIENT_SECRET = '0VHHAPVEUPRGEIG0RWXK3P4J3YW32P5ZTGS4WL2WQFFGPKQE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Successfully Logged-In')

Successfully Logged-In


In [22]:
Nebr_ungrp.loc[0]
neighborhood_latitude = np.float(Nebr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(Nebr_ungrp.loc[0,['Longitude']].values)

###  Now, let's get the top 100 venues that are in Harbour Square Park within a radius of 500 meters.

###### First, let's create the GET request URL. Name the URL url.

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [ ]:
results = requests.get(url).json()
results

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/amine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Mr. Jerk,Caribbean Restaurant,43.667328,-79.373389
1,Cranberries,Diner,43.667843,-79.369407
2,Murgatroid,Restaurant,43.667381,-79.369311
3,F'Amelia,Italian Restaurant,43.667536,-79.368613
4,Absolute Bakery & Café,Bakery,43.667469,-79.369277


And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

31 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
toronto_venues = getNearbyVenues(names=Nebr_ungrp['Neighborhood'],
                                   latitudes=Nebr_ungrp['Latitude'],
                                   longitudes=Nebr_ungrp['Longitude']
                                  )

St. James Town

The Beaches

Berczy Park

Christie

Dufferin, Dovercourt Village

Studio District

Lawrence Park

Roselawn

Davisville North

Parkdale, Roncesvalles

Davisville

University of Toronto, Harbord

Runnymede, Swansea

Rosedale

Church and Wellesley



In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(742, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. James Town\n,43.669403,-79.372704,Mr. Jerk,43.667328,-79.373389,Caribbean Restaurant
1,St. James Town\n,43.669403,-79.372704,Cranberries,43.667843,-79.369407,Diner
2,St. James Town\n,43.669403,-79.372704,Murgatroid,43.667381,-79.369311,Restaurant
3,St. James Town\n,43.669403,-79.372704,F'Amelia,43.667536,-79.368613,Italian Restaurant
4,St. James Town\n,43.669403,-79.372704,Absolute Bakery & Café,43.667469,-79.369277,Bakery


In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park\n,100,100,100,100,100,100
Christie\n,57,57,57,57,57,57
Church and Wellesley\n,73,73,73,73,73,73
Davisville\n,40,40,40,40,40,40
Davisville North\n,40,40,40,40,40,40
"Dufferin, Dovercourt Village\n",43,43,43,43,43,43
Lawrence Park\n,49,49,49,49,49,49
"Parkdale, Roncesvalles\n",30,30,30,30,30,30
Rosedale\n,3,3,3,3,3,3


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 173 uniques categories.


## 3. Analyze Each Neighborhood

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Sake Bar,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Track,Trail,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,St. James Town\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. James Town\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. James Town\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,St. James Town\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,St. James Town\n,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [34]:
toronto_onehot.shape

(742, 174)

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Sake Bar,Salon / Barbershop,Sandwich Place,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Track,Trail,Tree,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park\n,0.000000,0.000000,0.010000,0.010000,0.02,0.000000,0.000000,0.000000,0.020000,0.01,0.030000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.010000,0.00,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.02,0.000000,0.000000,0.01,0.000000,0.020000,0.080000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.020000,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.01,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.010,0.000000,0.010000,0.030000,0.00,0.000000,0.000000,0.000000,0.040000,0.010000,0.010000,0.000,0.010,0.050000,0.040000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.01,0.01,0.010000,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.00,0.01,0.000000,0.010000,0.01,0.000000,0.010000,0.000000,0.000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
1,Christie\n,0.000000,0.000000,0.017544,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0

In [36]:
toronto_grouped.shape

(15, 174)

Lets Check top Venues

In [37]:
Top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(Top_venues))
    print('\n')

----Berczy Park
----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.06
2  Italian Restaurant  0.05
3          Restaurant  0.05
4               Hotel  0.04


----Christie
----
               venue  freq
0  Korean Restaurant  0.25
1        Coffee Shop  0.05
2      Grocery Store  0.04
3                Pub  0.04
4       Cocktail Bar  0.04


----Church and Wellesley
----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.05
2     Sushi Restaurant  0.04
3              Gay Bar  0.04
4           Restaurant  0.04


----Davisville
----
                venue  freq
0  Italian Restaurant  0.10
1    Sushi Restaurant  0.10
2         Coffee Shop  0.08
3                Park  0.05
4   Convenience Store  0.05


----Davisville North
----
                venue  freq
0  Italian Restaurant  0.10
1    Sushi Restaurant  0.10
2         Coffee Shop  0.08
3                Park  0.05
4   Convenience Store  0.05


----Dufferin, Dovercourt Village
--

In [38]:
def return_most_common_venues(row, Top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venues]

In [39]:
Top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Top_venues):
    try:
        columns.append('{}{} Popular Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Popular Venues'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], Top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Berczy Park\n,Coffee Shop,Café,Italian Restaurant,Restaurant,Japanese Restaurant,Hotel,Gym,Seafood Restaurant,Bakery,Beer Bar
1,Christie\n,Korean Restaurant,Coffee Shop,Dessert Shop,Ice Cream Shop,Pub,Cocktail Bar,Grocery Store,Mexican Restaurant,Sandwich Place,Indian Restaurant
2,Church and Wellesley\n,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Café,Hotel,Diner,Yoga Studio,Mediterranean Restaurant
3,Davisville\n,Italian Restaurant,Sushi Restaurant,Coffee Shop,Park,Indian Restaurant,Pub,Convenience Store,Deli / Bodega,Falafel Restaurant,Middle Eastern Restaurant
4,Davisville North\n,Italian Restaurant,Sushi Restaurant,Coffee Shop,Park,Indian Restaurant,Pub,Convenience Store,Deli / Bodega,Falafel Restaurant,Middle Eastern Restaurant
5,"Dufferin, Dovercourt Village\n",Bar,Bakery,Café,Coffee Shop,Sandwich Place,Vietnamese Restaurant,Restaurant,Cocktail Bar,Beer Store,Mexican Restaurant
6,Lawrence Park\n,Sushi Restaurant,Italian Restaurant,Bakery,Coffee Shop,Fast Food Restaurant,Asian Restaurant,Bank,Pub,Pizza Place,Cosmetics Shop
7,"Parkdale, Roncesvalles\n",Pharmacy,Coffee Shop,Eastern European Restaurant,Café,Bakery,Convenience Store,Beach,Pie Shop,Playground,Restaurant
8,Rosedale\n,Playground,Bike Trail,Park,Eastern European Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,Roselawn\n,Italian Restaurant,Coffee Shop,Gym,Park,Diner,Bakery,Restaurant,Wine Bar,Pub,Food & Drink Shop


## 4. Cluster Neighborhoodsusing K-Mean

In [50]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[0 3 0 4 4 2 4 2 1 0 2 2 0 0 2]


In [54]:
toronto_merged = Nebr_ungrp
print(toronto_merged.shape)
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

(15, 6)
(15,)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,M5C\n,Downtown Toronto\n,St. James Town\n,43.669403,-79.372704,0,0,Coffee Shop,Pizza Place,Grocery Store,Café,Italian Restaurant,Pharmacy,Caribbean Restaurant,Restaurant,Hotel,Pet Store
1,M4E\n,East Toronto\n,The Beaches\n,43.671024,-79.296712,3,3,Beach,Thai Restaurant,Park,Japanese Restaurant,Pizza Place,Bar,Breakfast Spot,Tea Room,Greek Restaurant,Dance Studio
2,M5E\n,Downtown Toronto\n,Berczy Park\n,43.647984,-79.375396,0,0,Coffee Shop,Café,Italian Restaurant,Restaurant,Japanese Restaurant,Hotel,Gym,Seafood Restaurant,Bakery,Beer Bar
3,M6G\n,Downtown Toronto\n,Christie\n,43.664111,-79.418405,4,4,Korean Restaurant,Coffee Shop,Dessert Shop,Ice Cream Shop,Pub,Cocktail Bar,Grocery Store,Mexican Restaurant,Sandwich Place,Indian Restaurant
4,M6H\n,West Toronto\n,"Dufferin, Dovercourt Village\n",43.660202,-79.435719,4,4,Bar,Bakery,Café,Coffee Shop,Sandwich Place,Vietnamese Restaurant,Restaurant,Cocktail Bar,Beer Store,Mexican Restaurant


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
0,Downtown Toronto\n,0,0,Coffee Shop,Pizza Place,Grocery Store,Café,Italian Restaurant,Pharmacy,Caribbean Restaurant,Restaurant,Hotel,Pet Store
2,Downtown Toronto\n,0,0,Coffee Shop,Café,Italian Restaurant,Restaurant,Japanese Restaurant,Hotel,Gym,Seafood Restaurant,Bakery,Beer Bar
9,West Toronto\n,0,0,Pharmacy,Coffee Shop,Eastern European Restaurant,Café,Bakery,Convenience Store,Beach,Pie Shop,Playground,Restaurant
12,West Toronto\n,0,0,Coffee Shop,Café,Bakery,Bank,Pizza Place,Gourmet Shop,Dance Studio,Sandwich Place,Restaurant,Gym
13,Downtown Toronto\n,0,0,Playground,Bike Trail,Park,Eastern European Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 2

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
8,Central Toronto\n,1,1,Italian Restaurant,Sushi Restaurant,Coffee Shop,Park,Indian Restaurant,Pub,Convenience Store,Deli / Bodega,Falafel Restaurant,Middle Eastern Restaurant


#### Cluster 3

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
5,East Toronto\n,2,2,Coffee Shop,Pizza Place,Bar,Vegetarian / Vegan Restaurant,Café,Clothing Store,Thai Restaurant,Cosmetics Shop,French Restaurant,Movie Theater
7,Central Toronto\n,2,2,Italian Restaurant,Coffee Shop,Gym,Park,Diner,Bakery,Restaurant,Wine Bar,Pub,Food & Drink Shop
10,Central Toronto\n,2,2,Italian Restaurant,Sushi Restaurant,Coffee Shop,Park,Indian Restaurant,Pub,Convenience Store,Deli / Bodega,Falafel Restaurant,Middle Eastern Restaurant
11,Downtown Toronto\n,2,2,Café,Coffee Shop,Restaurant,Bookstore,Japanese Restaurant,Gym,Yoga Studio,Bakery,Bubble Tea Shop,Beer Store
14,Downtown Toronto\n,2,2,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Café,Hotel,Diner,Yoga Studio,Mediterranean Restaurant


#### Cluster 4

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
1,East Toronto\n,3,3,Beach,Thai Restaurant,Park,Japanese Restaurant,Pizza Place,Bar,Breakfast Spot,Tea Room,Greek Restaurant,Dance Studio


#### Cluster 5

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Popular Venues,2nd Popular Venues,3rd Popular Venues,4th Popular Venues,5th Popular Venues,6th Popular Venues,7th Popular Venues,8th Popular Venues,9th Popular Venues,10th Popular Venues
3,Downtown Toronto\n,4,4,Korean Restaurant,Coffee Shop,Dessert Shop,Ice Cream Shop,Pub,Cocktail Bar,Grocery Store,Mexican Restaurant,Sandwich Place,Indian Restaurant
4,West Toronto\n,4,4,Bar,Bakery,Café,Coffee Shop,Sandwich Place,Vietnamese Restaurant,Restaurant,Cocktail Bar,Beer Store,Mexican Restaurant
6,Central Toronto\n,4,4,Sushi Restaurant,Italian Restaurant,Bakery,Coffee Shop,Fast Food Restaurant,Asian Restaurant,Bank,Pub,Pizza Place,Cosmetics Shop


<hr>

### Thanks for reading this project!

Notebook created by: <a href = "https://www.linkedin.com/in/amine-mahmoud/">Amine Mahmoud</a>